In [ ]:
import os
import shutil
import pickle
import subprocess
from pathlib import Path
from typing import Literal, List

import torch
import numpy as np
import trimesh
import matplotlib.pyplot as plt
from trimesh.voxel import VoxelGrid
from mpl_toolkits.mplot3d import Axes3D

## Redes Neuronales Recurrentes: Clasificación de Models CAD

En este cuaderno de Jupyer abordaremos el problema de entrenar una red neuronal convolucional tridimensional (3D-CNN) para clasificar modelos de objetos CAD provenientes del conjunto de datos ModelNet10. Utilizaremos la biblioteca trimesh de Python para manipular estos modelos, y la herramienta binvox para transformarlos en una representación voxelizada adecuada para su procesamiento con la 3D-CNN. Al final, evaluaremos y visualizaremos los resultados utilizando matplotlib.

1. Construcción de conjunto de datos ModelNet10Binvox
    - Explicación del conjunto de entrenmiento.
    - Lectura del conjunto y procesamiento de conjunto de datos.
    - Voxelización utilizando Binvox.
2. Entrenamiento de Modelo
    - Construcción de conjunto de datos para entrenamiento.
    - Definición de DataLoader y TensorDataset.
    - Entrenamiento registrando la exactitud y la perdida.
3. Análisis de resultados
    - Evaluación en el conjunto de pruebas para analizar el poder de generalización del modelo.
    - Demostración de algunas inferencias utilizando matplotlib para renderizar los modelos.

# 1. Construcción de conjunto de datos ModelNet10Binvox

La estructura de directorio de ModelNet10 se compone de 10 carpetas representando cada categoría de objeto. Dentro de cada una de estas carpetas se encuentras otras dos con los datos de entrenamiento y prueba repectivamente. Las diez categorías del conjunto de datos son bathtub, bed, chair, desk, dresser, monitor, night_stand, sofa, table y toilet.

- ModelNet10
    - bathtub
        - test
            - ...
        - train
            - ...
    - bed
        - test
            - ...
        - train
            - ...
    - ...

Vamos a construir un conjunto de datos ModelNet10Binvox que va a contener los modelos voxelizados. Su estructua va a ser más sencilla y solo tendrá dos directorios uno con los modelos de entrenamiento y otro con los de prueba. Las clases de los objetos se pueden distinguir todavía con el nombre del archivo.


- ModelNet10Binvox
    - train
        - ...
    - test
        - ...


In [ ]:
class BinvoxDatasetBuilder:
    def __init__(self, input_directory: str, output_directory: str) -> None:
        """ Clase para generar el conjunto ModelNet10Binvox.
        :param input_directory: Ruta del directorio que contiene los conjuntos de datos de ModelNet.
        :param output_directory: Ruta del directorio donde se almacenarán los conjuntos de datos procesados.
            El directorio debe de existir.
        """
        self._input_directory = input_directory
        self._output_directory = output_directory

    def execute(self) -> None:
        """Genera el conjunto de datos."""
        for split in ["train", "test"]:
            file_paths = self._list_dataset_file_paths(split)
            for path in file_paths:
                self._process_binvox(path, os.path.join(self._output_directory, split))

    def _list_dataset_file_paths(self, split: Literal["train", "test"]) -> List[str]:
        """Enumera las rutas absolutas de los archivos del conjunto de datos de entrenamiento o pruebas del
        dataset "ModelNet10".

        :param dataset_directory_path: Ruta del directorio que contiene los conjuntos de datos.
        :param split: Define si se deben listar los archivos del conjunto de datos de entrenamiento o prueba.
        :return: Una lista de cadenas de texto donde cada cadena es una ruta a un archivo en el conjunto de datos
            solicitado.
        """
        files_paths = []
        objects_paths = [p for p in Path(self._input_directory).iterdir() if p.is_dir()]
        for op in objects_paths:
            paths = [str(path) for path in Path(op / split).iterdir() if path.is_file()]
            files_paths.extend(paths)
        return files_paths

    def _process_binvox(self, file_path: str, output_dir: str) -> VoxelGrid:
        """Procesa un modelo cad con binvox y lo mueve al directorio de salida especificado.

        :param file_path: Ruta al archivo con extensión .off a procesar.
        :param output_dir: Ruta del directorio donde se almacenará el archivo .binvox procesado.
        """
        output_file_name = os.path.splitext(os.path.basename(file_path))[0] + ".binvox"
        default_output_path = os.path.join(os.path.dirname(file_path), output_file_name)
        output_path = os.path.join(output_dir, output_file_name)
        cmd = ["/ruta/hacia/binvox", "-d", "32", "-cb", file_path]
        output = subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        shutil.move(default_output_path, output_path)


In [ ]:
# Uso de la clase para generar conjunto ModelNet10Binvox
dataset_builder = BinvoxDatasetBuilder(
    input_directory="/ruta/hacia/ModelNet10",
    output_directory="/ruta/hacia/ModelNet10Binvox"
)

# 2. Entrenamiento de Modelo

El conjunto ModelNet10Binvox nos ayuda a tener una referencia de los modelos voxelizados; sin embargo, no se puede usar directamente para entrena la red convolucional, para esto, tenemos que convertir los modelos de formato .binvox a tendores.

El objetivo es crear 4 tensores:
- train_matrix: Contiene la representación tensorial de los modelos, tiene dimensiones (3991, 38, 38, 38)
- train_labels: Contiene la representación numérica de la clase del modelo, tiene dimensiones (3991, )
- test_matrix: Contiene la representación tensorial de los modelos, tiene dimensiones (908, 38, 38, 38)
- test_labels: Contiene la representación numérica de la clase del modelo, tiene dimensiones (908, )

Los modelos de binvox tienen dimensiones (32, 32, 32) pero aquí los convertimos a (38, 38, 38) en donde se les agrega padding lleno de zeros en las tres dimensiones con el objetivo de no perder información de las esquinas.

In [ ]:
# La variable "categories" nos ayudará a convertir una categoría de su representación numérica a su representación
# como palabra y viceversa. La conversión se realiza utilizando el índice de la palabra e.g. la representación
# numérica de "table" es 8. La palabra "night" se refiere a "night_stand"
categories = ["bathtub", "bed", "chair", "desk", "dresser", "monitor", "night", "sofa", "table", "toilet"]

# Construcción del conjunto de entrenamiento
binxov_path = "/ruta/hacia/ModelNet10Binvox/train"
num_train_files = len(os.listdir(binxov_path))
train_matrix = torch.zeros((num_train_files, 38, 38, 38), dtype=torch.float32)
train_labels = torch.empty((num_train_files,), dtype=torch.uint8)
for i, file in enumerate(os.scandir(binxov_path)):
    print(f"i, file.path: {i}, {file.path}")
    with open(file.path, "rb") as f:
        voxel_grid = trimesh.exchange.binvox.load_binvox(f)
    train_labels[i] = categories.index(file.name.split("_")[0])
    train_matrix[i, 3:-3, 3:-3, 3:-3] = torch.tensor(voxel_grid.matrix)
torch.save(train_matrix, "/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Pytorch/train.pt")
torch.save(train_labels, "/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Pytorch/train_labels.pt")

# Construcción del conjunto de pruebas
binxov_path = "/ruta/hacia/ModelNet10Binvox/test"
num_test_files = len(os.listdir(binxov_path))
test_matrix = torch.zeros((num_test_files, 38, 38, 38), dtype=torch.float32)
test_labels = torch.empty((num_test_files,), dtype=torch.uint8)
for i, file in enumerate(os.scandir(binxov_path)):
    print(f"i, file.path: {i}, {file.path}")
    with open(file.path, "rb") as f:
        voxel_grid = trimesh.exchange.binvox.load_binvox(f)
    test_labels[i] = categories.index(file.name.split("_")[0])
    test_matrix[i, 3:-3, 3:-3, 3:-3] = torch.tensor(voxel_grid.matrix)
torch.save(test_labels, "/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Pytorch/test_labels.pt")
torch.save(test_matrix, "/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Pytorch/test.pt")

In [ ]:
# Cargar los datos precomputados
train_matrix = torch.load("/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Pytorch/train.pt")
train_labels = torch.load("/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Pytorch/train_labels.pt")
test_matrix = torch.load("/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Pytorch/test.pt")
test_labels = torch.load("/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Pytorch/test_labels.pt")

In [ ]:
print(f"train_matrix.shape: {train_matrix.shape}")
print(f"train_labels.shape: {train_labels.shape}")
print(f"test_matrix.shape: {test_matrix.shape}")
print(f"test_labels.shape: {test_labels.shape}")

train_matrix.shape: torch.Size([3991, 38, 38, 38])
train_labels.shape: torch.Size([3991])
test_matrix.shape: torch.Size([908, 38, 38, 38])
test_labels.shape: torch.Size([908])


In [ ]:
class CadNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Tensor de entrada tiene dimensiones (batch_size, 1 38, 38, 38)
        self.conv_layers = torch.nn.Sequential(
            torch.nn.Conv3d(
                in_channels=1,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ), # (batch_size, 32, 38, 38, 38)
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool3d(kernel_size=2, stride=2),  # (batch_size, 32, 19, 19, 19)

            torch.nn.Conv3d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1
            ),  # (batch_size, 64, 19, 19, 19)
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool3d(kernel_size=2, stride=2),  # (batch_size, 64, 9, 9, 9)
        )

        self.fc_layers = torch.nn.Sequential(
            torch.nn.Linear(64 * 9 * 9 * 9, 512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),
            torch.nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1) # flatten tensor
        x = self.fc_layers(x)
        return x  # (batch_size, 10)

In [ ]:
# Las convoluciones 3D esperan una entrada con dimensiones (batch_size, channels, depth, height, width), por lo que
# tenemos que agregar la dimensión "channels"
train_matrix = train_matrix.unsqueeze(1)
test_matrix = test_matrix.unsqueeze(1)
print(f"train_matrix.shape: {train_matrix.shape}")
print(f"test_matrix.shape: {test_matrix.shape}")

train_matrix.shape: torch.Size([3991, 1, 38, 38, 38])
test_matrix.shape: torch.Size([908, 1, 38, 38, 38])


In [ ]:
# Utilizamos TensorDataset para generar el conjunto de datos, de acuerdo a la documentación oficial "Cada muestra se
# recuperará indexando tensores a lo largo de la primera dimensión". También generamos los DataLoaders con un tamaño
# de bache de 64.
train_data = torch.utils.data.TensorDataset(train_matrix, train_labels)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

test_data = torch.utils.data.TensorDataset(test_matrix, test_labels)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
device = torch.device('cpu')
model = CadNet().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

loss_history = []
accuracy_history = []
for epoch in range(10):  # Se entrena por diez épocas
    # Correct y total se utilizand para calcular la exactitud
    correct = 0
    total = 0
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Propagación hacia adelante
        outputs = model(inputs)  # (batch_size, categories)
        loss = criterion(outputs, labels)  # Make sure labels are long type
        total_loss += loss

        # Calcular la exactitud
        # torch.max regresa una tupla de (valores, índices) en donde para cada fila especificada por
        # el argumento dimensión se regresa el valor máximo y los índices de estos.
        _, predicted = torch.max(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Retropropagación
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    accuracy = 100 * correct / total
    accuracy_history.append(accuracy)
    loss_history.append(total_loss / total)
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}, Accuracy: {accuracy}%")

torch.save(model.state_dict(), 'model_state_dict.pt')
with open("accuracy_history.pkl", "wb") as f:
    pickle.dump(accuracy_history, f)

Epoch 1, Loss: 0.3834688663482666, Accuracy: 80.20546229015284%
Epoch 2, Loss: 0.12099435180425644, Accuracy: 91.15509897268855%
Epoch 3, Loss: 0.11604078859090805, Accuracy: 93.43522926584816%
Epoch 4, Loss: 0.18583758175373077, Accuracy: 94.23703332498121%
Epoch 5, Loss: 0.07984792441129684, Accuracy: 95.26434477574543%
Epoch 6, Loss: 0.10109993815422058, Accuracy: 96.51716361814081%
Epoch 7, Loss: 0.0355650894343853, Accuracy: 96.4419944875971%
Epoch 8, Loss: 0.06436997652053833, Accuracy: 96.96817840140315%
Epoch 9, Loss: 0.19982895255088806, Accuracy: 97.76998246053621%
Epoch 10, Loss: 0.013670407235622406, Accuracy: 97.64470057629667%


# 3. Análisis de resultados

In [ ]:
# Cargar el modelo y el historial de la exactitud
model = CadNet()
model.load_state_dict(torch.load('model_state_dict.pt'))
with open("accuracy_history.pkl", "rb") as f:
    accuracy_history = pickle.load(f)

In [ ]:
# Graficar la progresión de la exactitud
epochs = range(1, len(accuracy_history) + 1)
plt.figure(figsize=(10, 6))
plt.plot(epochs, accuracy_history, "b", label="Training accuracy")
plt.plot(epochs, accuracy_history, "b", label="Training accuracy")
plt.title("Exactitud por época", fontsize=16)
plt.xlabel("Época", fontsize=14)
plt.ylabel("Exactitud", fontsize=14)
plt.grid()
plt.show()

In [ ]:
# Obtener exactitud en conjunto de pruebas
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:

        outputs = model(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

In [ ]:
def classify_model(model_path: str, model: torch.nn.Module) -> str:
    """Clasifica un objeto 3D utilizando un modelo preentrenado.

    Esta función carga los datos de voxel de un archivo .binvox, los prepara para la entrada al
    modelo proporcionado, realiza un pase hacia adelante del modelo, y devuelve la categoría del
    objeto 3D.

    :param model_path: Ruta al archivo .binvox que contiene los datos de voxel para el objeto 3D.
    :param model: El modelo PyTorch preentrenado a utilizar para la tarea de clasificación.
    :return: La categoría del objeto 3D, según la identifica el modelo.
    """
    with open(model_path, "rb") as f:
        voxel_data = trimesh.exchange.binvox.load_binvox(f).matrix
    input = torch.zeros((1, 1, 38 ,38, 38))
    input[0, 0, 3:-3, 3:-3, 3:-3] = torch.tensor(voxel_data)
    model.eval()
    with torch.no_grad():
        output = model(input)
        # torch.argmax devuelve los índices de los valores máximos de un tensor a lo largo de una dimensión.
        class_index = torch.argmax(output)
    categories = ["bathtub", "bed", "chair", "desk", "dresser", "monitor", "night", "sofa", "table", "toilet"]
    return categories[class_index.item()]


def plot_model(model_path: str) -> None:
    """Grafica el modelo voxelizado."""
    with open(model_path, "rb") as f:
        voxel_data = trimesh.exchange.binvox.load_binvox(f).matrix

    # Prepare some coordinates
    x, y, z = np.indices(np.array(voxel_data.shape) + 1)

    # Draw the voxels
    voxels = voxel_data

    # Choose colors for each voxel
    colors = np.empty(voxels.shape, dtype=object)
    colors[voxels] = 'blue'  # Color the True voxels

    # Plot everything
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.voxels(x, y, z, voxels, facecolors=colors, edgecolors='k', linewidth=0.5, alpha=0.8)


In [ ]:
model_path = "/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Binvox/test/bathtub_0107.binvox"
category = classify_model(model_path, model)
print(f"category: {category}")
plot_model(model_path)

In [ ]:
model_path = "/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Binvox/test/chair_0891.binvox"
category = classify_model(model_path, model)
print(f"category: {category}")
plot_model(model_path)

In [ ]:
model_path = "/home/kosmos/artur/unam/cad_object_recognition/ModelNet10Binvox/test/sofa_0757.binvox"
category = classify_model(model_path, model)
print(f"category: {category}")
plot_model(model_path)